In [56]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [57]:
import os 
import torch 
from torch import nn
import torch.nn.functional as F

import numpy as np 

import src.utils as utils 
import src.globals as globals
import src.data_handler as handling

from datasets import Dataset 

from tokenizers import BertWordPieceTokenizer


In [58]:
dataset_path = os.path.join(globals.DATA_FOLDER,'training_set.json')

In [59]:
model, vocab = utils.load_embedding_model()

In [60]:
for e in vocab :
    if vocab[e]==1012 :
        print(e)

vocab['include']

includes


488

In [61]:
squad_dataset = handling.RawSquadDataset(dataset_path)

df = squad_dataset.train_df.copy()

In [62]:
l = df['context'].apply(len).to_list()

m = np.mean(l)
m

754.3642164864896

In [63]:
df

,context_id,question_id,title,context,question,answer,label_char
0,0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,"(515, 541)"
1,0,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,"(188, 213)"
2,0,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,"(279, 296)"
3,0,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,"(381, 420)"
4,0,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,"(92, 126)"
...,...,...,...,...,...,...,...
87594,18890,5735d259012e2f140011a09d,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,Oregon,"(229, 235)"
87595,18890,5735d259012e2f140011a09e,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,Rangoon,"(414, 421)"
87596,18890,5735d259012e2f140011a09f,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,Minsk,"(476, 481)"
87597,18890,5735d259012e2f140011a0a0,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,1975,"(199, 203)"


In [64]:
s1 = "The sky is usually clear above the desert and the sunshine duration is extremely high everywhere in the Sahara. Most of the desert enjoys more than 3,600 h of bright sunshine annually or over 82% of the time and a wide area in the eastern part experiences in excess of 4,000 h of bright sunshine a year or over 91% of the time, and the highest values are very close to the theoretical maximum value. A value of 4,300 h or 98% of the time would be recorded in Upper Egypt (Aswan, Luxor) and in the Nubian Desert (Wadi Halfa). The annual average direct solar irradiation is around 2,800 kWh/(m2 year) in the Great Desert. The Sahara has a huge potential for solar energy production. The constantly high position of the sun, the extremely low relative humidity, the lack of vegetation and rainfall make the Great Desert the hottest continuously large area worldwide and certainly the hottest place on Earth during summertime in some spots. The average high temperature exceeds 38 °C (100.4 °F) - 40 °C (104 °F) during the hottest month nearly everywhere in the desert except at very high mountainous areas. The highest officially recorded average high temperature was 47 °C (116.6 °F) in a remote desert town in the Algerian Desert called Bou Bernous with an elevation of 378 meters above sea level. It's the world's highest recorded average high temperature and only Death Valley, California rivals it. Other hot spots in Algeria such as Adrar, Timimoun, In Salah, Ouallene, Aoulef, Reggane with an elevation between 200 and 400 meters above sea level get slightly lower summer average highs around 46 °C (114.8 °F) during the hottest months of the year. Salah, well known in Algeria for its extreme heat, has an average high temperature of 43.8 °C (110.8 °F), 46.4 °C (115.5 °F), 45.5 (113.9 °F). Furthermore, 41.9 °C (107.4 °F) in June, July, August and September. In fact, there are even hotter spots in the Sahara, but they are located in extremely remote areas, especially in the Azalai, lying in northern Mali. The major part of the desert experiences around 3 – 5 months when the average high strictly exceeds 40 °C (104 °F). The southern central part of the desert experiences up to 6 – 7 months when the average high temperature strictly exceeds 40 °C (104 °F) which shows the constancy and the length of the really hot season in the Sahara. Some examples of this are Bilma, Niger and Faya-Largeau, Chad. The annual average daily temperature exceeds 20 °C (68 °F) everywhere and can approach 30 °C (86 °F) in the hottest regions year-round. However, most of the desert has a value in excess of 25 °C (77 °F). The sand and ground temperatures are even more extreme. During daytime, the sand temperature is extremely high as it can easily reach 80 °C (176 °F) or more. A sand temperature of 83.5 °C (182.3 °F) has been recorded in Port Sudan. Ground temperatures of 72 °C (161.6 °F) have been recorded in the Adrar of Mauritania and a value of 75 °C (167 °F) has been measured in Borkou, northern Chad. Due to lack of cloud cover and very low humidity, the desert usually features high diurnal temperature variations between days and nights. However, it's a myth that the nights are cold after extremely hot days in the Sahara. The average diurnal temperature range is typically between 13 °C (55.4 °F) and 20 °C (68 °F). The lowest values are found along the coastal regions due to high humidity and are often even lower than 10 °C (50 °F), while the highest values are found in inland desert areas where the humidity is the lowest, mainly in the southern Sahara. Still, it's true that winter nights can be cold as it can drop to the freezing point and even below, especially in high-elevation areas. "
s2 = "The sky is usually clear above the desert and the sunshine duration is extremely high everywhere in the Sahara. Most of the desert enjoys more than 3,600 h of bright sunshine annually or over 82% of the time and a wide area in the eastern part experiences in excess of 4,000 h of bright sunshine a year or over 91% of the time, and the highest values are very close to the theoretical maximum value. A value of 4,300 h or 98% of the time would be recorded in Upper Egypt (Aswan, Luxor) and in the Nubian Desert (Wadi Halfa). The annual average direct solar irradiation is around 2,800 kWh/(m2 year) in the Great Desert. The Sahara has a huge potential for solar energy production. The constantly high position of the sun, the extremely low relative humidity, the lack of vegetation and rainfall make the Great Desert the hottest continuously large area worldwide and certainly the hottest place on Earth during summertime in some spots. The average high temperature exceeds 38 °C (100.4 °F) - 40 °C (104 °F) during the hottest month nearly everywhere in the desert except at very high mountainous areas. The highest officially recorded average high temperature was 47 °C (116.6 °F) in a remote desert town in the Algerian Desert called Bou Bernous with an elevation of 378 meters above sea level. It's the world's highest recorded average high temperature and only Death Valley, California rivals it. Other hot spots in Algeria such as Adrar, Timimoun, In Salah, Ouallene, Aoulef, Reggane with an elevation between 200 and 400 meters above sea level get slightly lower summer average highs around 46 °C (114.8 °F) during the hottest months of the year. Salah, well known in Algeria for its extreme heat, has an average high temperature of 43.8 °C (110.8 °F), 46.4 °C (115.5 °F), 45.5 (113.9 °F). Furthermore, 41.9 °C (107.4 °F) in June, July, August and September. In fact, there are even hotter spots in the Sahara, but they are located in extremely remote areas, especially in the Azalai, lying in northern Mali. The major part of the desert experiences around 3 – 5 months when the average high strictly exceeds 40 °C (104 °F). The southern central part of the desert experiences up to 6 – 7 months when the average high temperature strictly exceeds 40 °C (104 °F) which shows the constancy and the length of the really hot season in the Sahara. Some examples of this are Bilma, Niger and Faya-Largeau, Chad. The annual average daily temperature exceeds 20 °C (68 °F) everywhere and can approach 30 °C (86 °F) in the hottest regions year-round. However, most of the desert has a value in excess of 25 °C (77 °F). The sand and ground temperatures are even more extreme. During daytime, the sand temperature is extremely high as it can easily reach 80 °C (176 °F) or more. A sand temperature of 83.5 °C (182.3 °F) has been recorded in Port Sudan. Ground temperatures of 72 °C (161.6 °F) have been recorded in the Adrar of Mauritania and a value of 75 °C (167 °F) has been measured in Borkou, northern Chad. Due to lack of cloud cover and very low humidity, the desert usually features high diurnal temperature variations between days and nights. However, it's a myth that the nights are cold after extremely hot days in the Sahara. The average diurnal temperature range is typically between 13 °C (55.4 °F) and 20 °C (68 °F). The lowest values are found along the coastal regions due to high humidity and are often even lower than 10 °C (50 °F), while the highest values are found in inland desert areas where the humidity is the lowest, mainly in the southern Sahara. Still, it's true that winter nights can be cold as it can drop to the freezing point and even below, especially in high-elevation areas."
s3 = "The sky is usually clear above the desert and the sunshine duration is extremely high everywhere in the Sahara. Most of the desert enjoys more than 3,600 h of bright sunshine annually or over 82% of the time and a wide area in the eastern part experiences in excess of 4,000 h of bright sunshine a year or over 91% of the time, and the highest values are very close to the theoretical maximum value. A value of 4,300 h or 98% of the time would be recorded in Upper Egypt (Aswan, Luxor) and in the Nubian Desert (Wadi Halfa). The annual average direct solar irradiation is around 2,800 kWh/(m2 year) in the Great Desert. The Sahara has a huge potential for solar energy production. The constantly high position of the sun, the extremely low relative humidity, the lack of vegetation and rainfall make the Great Desert the hottest continuously large area worldwide and certainly the hottest place on Earth during summertime in some spots. The average high temperature exceeds 38 °C (100.4 °F) - 40 °C (104 °F) during the hottest month nearly everywhere in the desert except at very high mountainous areas. The highest officially recorded average high temperature was 47 °C (116.6 °F) in a remote desert town in the Algerian Desert called Bou Bernous with an elevation of 378 meters above sea level. It's the world's highest recorded average high temperature and only Death Valley, California rivals it. Other hot spots in Algeria such as Adrar, Timimoun, In Salah, Ouallene, Aoulef, Reggane with an elevation between 200 and 400 meters above sea level get slightly lower summer average highs around 46 °C (114.8 °F) during the hottest months of the year. Salah, well known in Algeria for its extreme heat, has an average high temperature of 43.8 °C (110.8 °F), 46.4 °C (115.5 °F), 45.5 (113.9 °F). Furthermore, 41.9 °C (107.4 °F) in June, July, August and September. In fact, there are even hotter spots in the Sahara, but they are located in extremely remote areas, especially in the Azalai, lying in northern Mali. The major part of the desert experiences around 3 – 5 months when the average high strictly exceeds 40 °C (104 °F). The southern central part of the desert experiences up to 6 – 7 months when the average high temperature strictly exceeds 40 °C (104 °F) which shows the constancy and the length of the really hot season in the Sahara. Some examples of this are Bilma, Niger and Faya-Largeau, Chad. The annual average daily temperature exceeds 20 °C (68 °F) everywhere and can approach 30 °C (86 °F) in the hottest regions year-round. However, most of the desert has a value in excess of 25 °C (77 °F). The sand and ground temperatures are even more extreme. During daytime, the sand temperature is extremely high as it can easily reach 80 °C (176 °F) or more. A sand temperature of 83.5 °C (182.3 °F) has been recorded in Port Sudan. Ground temperatures of 72 °C (161.6 °F) have been recorded in the Adrar of Mauritania and a value of 75 °C (167 °F) has been measured in Borkou, northern Chad. Due to lack of cloud cover and very low humidity, the desert usually features high diurnal temperature variations between days and nights. However, it's a myth that the nights are cold after extremely hot days in the Sahara. The average diurnal temperature range is typically between 13 °C (55.4 °F) and 20 °C (68 °F). The lowest values are found along the coastal regions due to high humidity and are often even lower than 10 °C (50 °F), while the highest values are found in inland desert areas where the humidity is the lowest, mainly in the southern Sahara. Still, it's true that winter nights can be cold as it can drop to the freezing point and even below, especially in high-elevation areas."
q = 'my question my question my question my question'
df_custom = {'context_id':18890,'question_id':'il_mio_custom','title':'inventato','context':s1+s2+s3,'question':q,'answer':'risposta mia','label_char':(10421,10435)}

df = df.append(df_custom,ignore_index=True)

In [65]:
df

,context_id,question_id,title,context,question,answer,label_char
0,0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,"(515, 541)"
1,0,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,"(188, 213)"
2,0,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,"(279, 296)"
3,0,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,"(381, 420)"
4,0,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,"(92, 126)"
...,...,...,...,...,...,...,...
87595,18890,5735d259012e2f140011a09e,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,Rangoon,"(414, 421)"
87596,18890,5735d259012e2f140011a09f,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,Minsk,"(476, 481)"
87597,18890,5735d259012e2f140011a0a0,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,1975,"(199, 203)"
87598,18890,5735d259012e2f140011a0a1,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",What is KMC an initialism of?,Kathmandu Metropolitan City,"(0, 27)"


In [66]:

hf_dataset = Dataset.from_pandas(df)

In [67]:
vocab_path = os.path.join(globals.DATA_FOLDER,'bert-base-uncased-vocab.txt')

In [68]:
wp_tokenizer = BertWordPieceTokenizer(vocab_path, lowercase=True)
wp_tokenizer.enable_padding(direction="right", pad_type_id=1)
wp_tokenizer.enable_truncation(512,strategy='only_second',stride = 25)

In [69]:
from tokenizers import Encoding

def transform(batch):

    encodings: list[Encoding] = wp_tokenizer.encode_batch(list(zip(batch['question'],batch['context'])))

    starts = list(map(lambda x: x[0],batch['label_char']))
    ends = list(map(lambda x: x[1],batch['label_char']))

    out = []
    replaced = []
    not_replaced = []
    for i,e in enumerate(encodings) :
        if e.char_to_token(starts[i],1) is None or e.char_to_token(ends[i]-1,1) is None :
            out.append(i)
            flag = 0
            for k,o in enumerate(e.overflowing) :
                if o.char_to_token(starts[i],1) is not None and o.char_to_token(ends[i]-1,1) is not None :
                    encodings[i] = o
                    replaced.append(k)
                    flag = 1
                    break
            if flag==0 :
                not_replaced.append(i)
    
    for idx in not_replaced:
        encodings.pop(idx)
        starts.pop(idx)
        ends.pop(idx)
        batch['context'].pop(idx)
        batch['answer'].pop(idx)


    # for i, (s_t, e_t) in enumerate(zip(token_start,token_end)):
    #     if (s_t is None) or (e_t is None):
    #         for e in 


    encodings = {
        'ids': [e.ids for e in encodings],
        'overflowing': [e.overflowing for e in encodings],
        'special_tokens_mask': [e.special_tokens_mask for e in encodings],
        #'mask': [e.attention_mask for e in encodings],
        'offsets': [e.offsets for e in encodings], 
        #'sequence_ids': [e.sequence_ids for e in encodings],
        'type_ids': [e.type_ids for e in encodings],
        'tokens':[e.tokens for e in encodings],
        'label_token_start': [e.char_to_token(starts[i],1) for i,e in enumerate(encodings)],
        'label_token_end': [e.char_to_token(ends[i]-1,1) for i,e in enumerate(encodings)],
        'context_text': batch['context'],
        'question_text': batch['question'],
        'answer_text': batch['answer'],
        'char_start' : starts,
        'char_end': ends,
        # 'out': out,
        # 'replaced' : replaced,
        # 'not_replaced' : not_replaced
    }

    return encodings

hf_dataset.set_transform(transform,output_all_columns=False)

In [ ]:
encodings: list[Encoding] = wp_tokenizer.encode_batch(list(zip(df['question'],df['context'])))

starts = list(map(lambda x: x[0],df['label_char']))
ends = list(map(lambda x: x[1],df['label_char']))

answer_not_in_first_512 = []
replaced = []
not_replaced = []
for i,e in enumerate(encodings) :
    if e.char_to_token(starts[i],1) is None or e.char_to_token(ends[i]-1,1) is None :
        answer_not_in_first_512.append(i)
        flag = 0
        for k,o in enumerate(e.overflowing) :
            if o.char_to_token(starts[i],1) is not None and o.char_to_token(ends[i]-1,1) is not None :
                encodings[i] = o
                replaced.append(k)
                flag = 1
                break
        if flag==0 :
            not_replaced.append(i)

for idx in not_replaced:
    encodings.pop(idx)
    starts.pop(idx)
    ends.pop(idx)
    df['context'].pop(idx)
    df['answer'].pop(idx)

In [ ]:
row_errors = []
for idx in range(df.shape[0]):

    ex = hf_dataset[idx]

    offsets = np.array(ex['offsets'])[np.array(ex['type_ids'], dtype=bool)]

    start_token = ex['label_token_start']
    end_token = ex['label_token_end']
    start_char = ex['offsets'][start_token][0]
    end_char = ex['offsets'][end_token][1]

    start_c = ex['char_start']
    end_c = ex['char_end']

    if ex['context_text'][start_char:end_char] == ex['answer_text']:
        pass
    else:
        row_errors.append("DIFFERENT: " + str(idx))

    starts, ends = zip(*offsets)

    try :
        start_idx = starts.index(start_c)
    except :
        row_errors.append("NO start: " + str(idx))

    try: 
        end_idx = ends.index(end_c)
    except :
        row_errors.append("NO end: " + str(idx))

print(row_errors)

In [ ]:
answer_not_in_first_512

In [51]:
print(hf_dataset[50763, 50764])


{'ids': [[101, 2054, 1005, 1055, 2178, 2171, 2005, 1996, 3347, 1011, 12788, 1011, 18929, 1029, 102, 15946, 9693, 1999, 6489, 1010, 2122, 2220, 2225, 2075, 4580, 9693, 2020, 2048, 1011, 4403, 9693, 2007, 6357, 18929, 2015, 2127, 1038, 1012, 1043, 1012, 16983, 4168, 2764, 1037, 13618, 3347, 12788, 18929, 1012, 1996, 2236, 3751, 2194, 2211, 4975, 2093, 1011, 4403, 15946, 9693, 1999, 6607, 1012, 2011, 6306, 1010, 2236, 3751, 1998, 2225, 2075, 4580, 2772, 1037, 2892, 1011, 13202, 3820, 2005, 1996, 3347, 1011, 12788, 1011, 18929, 2640, 1010, 2101, 2170, 1996, 18197, 1011, 7980, 18929, 1012, 15946, 5013, 8377, 8577, 2013, 2122, 21644, 1998, 15463, 2020, 2107, 2008, 1037, 2531, 15149, 1006, 6522, 1007, 15946, 5013, 2747, 2038, 1996, 2168, 15986, 9646, 2004, 1037, 1021, 1012, 1019, 6522, 5013, 1999, 6347, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
print(np.array(hf_dataset[6735]['offsets'])[np.array(hf_dataset[6735]['type_ids'], dtype=bool)])

In [ ]:
np.where(hf_dataset[6735]['type_ids'], hf_dataset[6735]['offsets'], (-1,-1))

In [ ]:
print(hf_dataset[6735])

### oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo

In [ ]:
a = hf_dataset[0:6]['type_ids'] & ~hf_dataset[0:6]['special_tokens_mask']

In [ ]:
x_type_ids = torch.tensor([[0,0,0,0,1,1,1,1,1],[0,0,0,1,1,1,1,1,1]])
y_special_t= torch.tensor([[1,0,0,1,0,0,0,1,1],[1,0,1,0,0,0,0,0,1]])

b = x_type_ids & ~y_special_t
c = torch.tensor([[5.0,7,3,6,1,0,4,7,8],[2,5,3,8,9,0,3,6,8]])
c.masked_fill(b==0,float("-inf"))

In [ ]:
# print(hf_dataset[87599]['offsets'])
enc = hf_dataset[87599]['overflowing']

for i,e in enumerate(enc) :
    print(i+1,':', e.offsets)

In [ ]:
c_s = hf_dataset[60023]['context_text'].find('Still')
c_e = hf_dataset[60023]['context_text'].find('winter')

In [ ]:
print(hf_dataset[60023]['overflowing'])

In [ ]:
e = hf_dataset[60023]['overflowing'][0]

t_s = e.char_to_token(c_s,2)
t_e = e.char_to_token(c_e,2)

offsets = e.offsets

In [ ]:
c_s_p = offsets[t_s][0]
c_e_p = offsets[t_e][1]

hf_dataset[60023]['context_text'][c_s_p:c_e_p]

In [ ]:
e.overflowing

In [ ]:
[hf_dataset[60022]['tokens'].index('##urn'))
# print(hf_dataset[60023,60022]['context_text'])

In [ ]:
ex = hf_dataset[60022]
index = ex['tokens'].index('worldwide')
s= ex['offsets'][index][0]
e= ex['offsets'][index][1]
ex['context_text'][s:e]



In [ ]:
ex = hf_dataset[57912]
start_token = ex['label_token_start']
end_token = ex['label_token_end']
start_char = ex['offsets'][start_token][0]
end_char = ex['offsets'][end_token][1]

print(start_char)
print(end_char)

ex['context_text'][start_char:end_char]
ex['context_text'][ex['char_start']:ex['char_end']]
ex['answer_text']

len(ex['context_ids']) == len(ex['context_tokens'])
